In [6]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import dlib
import numpy as np
from imutils import face_utils
from torchvision import transforms
import glob
%matplotlib inline
import pandas as pd

font = cv2.FONT_HERSHEY_SIMPLEX

In [7]:
#torch.cuda.is_available()
device = torch.device('cpu')#'cuda' if torch.cuda.is_available else 'cpu'
device

device(type='cpu')

In [8]:
#mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20, device = device)
#image_size=240, margin=0, min_face_size=20, device= device)
model = torch.hub.load('ultralytics/yolov5', 'custom', path='face_detection_yolov5s.pt',device=device)
resnet = InceptionResnetV1(pretrained='vggface2', device= device).eval()

dataset=datasets.ImageFolder('lfw_split/train/')
print(len(dataset))
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} 
def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] 
name_list = [] 
embedding_list = [] 



URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [ ]:

for imgs, idx in loader:

    transform1 = transforms.ToTensor()
    tensor = transform1(imgs)
    t = transforms.functional.adjust_gamma(tensor, gamma=2, gain=3)

    transform2 = transforms.ToPILImage()

    # convert the tensor to PIL image using above transform
    imgss = transform2(t)

    img =  np.array(imgss)
    

    results = model(img)
    faces = results.pandas().xyxy[0]
    if faces.empty:
          continue
    print(faces)
    faces = faces.sort_values(by='confidence', ascending=False)
    x = round(faces['xmin'][0])
    y = round(faces['ymin'][0])
    w = round(faces['xmax'][0])
    h = round(faces['ymax'][0])
    img_ = img[y:h, x:w]
   # print(type(img_))
   # print(img_.shape)

    #img_padding = np.pad(img_, ((100, 100), (200, 200), (0,0)), constant_values=0)

   # print(img)
   # face, prob = mtcnn(img_padding, return_prob=True) 
    new_image = Image.fromarray(img_)   
    tensor_input = transform1(new_image)
    
    confidence = faces['confidence'][0]
    if tensor_input: #is not None and prob>0.90:
        emb = resnet(face.unsqueeze(0)) 
        embedding_list.append(emb.detach()) 
        name_list.append(idx_to_class[idx]) 





print(len(face_list))
print(len(name_list))
print(len(embedding_list))


In [ ]:
data = [embedding_list, name_list]
print(data[1])
torch.save(data, 'data.pt') 

In [ ]:

def face_match(img_path, data_path): 
    img_s = Image.open(img_path)
 
    transform1 = transforms.ToTensor()
    tensor = transform1(img_s)
    t = transforms.functional.adjust_gamma(tensor, gamma=2, gain=3)
    transform2 = transforms.ToPILImage()
    imgss = transform2(t)
    img =  np.array(imgss)

    results = model(img)
    faces = results.pandas().xyxy[0]
    if faces.empty:
        img_ = img
    else:
        faces = faces.sort_values(by='confidence', ascending=False)
        x = round(faces['xmin'][0])
        y = round(faces['ymin'][0])
        w = round(faces['xmax'][0])
        h = round(faces['ymax'][0])
        img_ = img[y:h, x:w]
    
    #padding
    new = Image.fromarray(img_)
    img_padding = np.pad(img_, ((100, 100), (200, 200), (0,0)), constant_values=0)
    
    img_pil = Image.fromarray(img_padding)
    face, prob = mtcnn(img_pil, return_prob=True) 
    
    if face is None:
        emb = transform1(new.resize((240,240)).unsqueeze(0)).detach() 

    else:
        emb = resnet(face.unsqueeze(0)).detach() 


    saved_data = torch.load('data.pt') 
    embedding_list = saved_data[0] 
    name_list = saved_data[1] 
    dist_list = [] 
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
    
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min].replace('_',' '), min(dist_list))

image_list = []
y_real = []
for filename in glob.glob('lfw_split/test/*/*.jpg'): #assuming gif
    image_list.append(filename)
    y_real.append(filename.split('\\')[1].split('0')[0].replace('_','').replace('.jpg',''))
    
predict_y = []
count = 1
for i in image_list:   
    result = face_match(i, 'data.pt')
    predict_y.append(result[0].replace(' ',''))
    count =count + 1
    print('Face matched with: (',result[0], ') With distance: ',result[1])
    

In [ ]:
print(len(y_real))
print(len(predict_y))
print(type(y_real))



In [ ]:
accuracy_marker = []
conf_y_real = []
conf_y_pred = []
marker = 0
for i in range(0,len(y_real)):
   # print(y[i])
   # print(predict_y[i])
    conf_y_real.append(0)
    if (y_real[i] == predict_y[i]) is True:
        accuracy_marker.append(0)
        conf_y_pred.append(0)
        marker +=1
    else:
        accuracy_marker.append(1)
        conf_y_pred.append(1)

    
print("accuracy:",marker/len(accuracy_marker))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt

confusion_matrix = confusion_matrix(conf_y_real,conf_y_pred)

sn.heatmap(confusion_matrix, annot=True)
plt.show()

# haraa cascade with mtcnn

#### accuracy : 77%

#### problem : haara is not good for faces except frontFaces

# yolo with mtcnn

#### accuracy : 80%


# yolo with resnet

#### accuracy : 94.73%